In [1]:
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, START, END
from typing import TypedDict,Annotated
import operator
from dotenv import load_dotenv
from pydantic import BaseModel,Field
load_dotenv()

True

In [ ]:
class EvalSchema(BaseModel): # This pydantic class defines the schema of the output from the LLM , only certain LLMs can support this feature 
    random_number : int = Field(description="A random generated number between 10 and 100",ge=10,le=100)

In [3]:
model=ChatOpenAI(model="gpt-4o-mini")
struct_model=model.with_structured_output(EvalSchema)

In [ ]:
class randomState(TypedDict):
    first_random_number: int
    second_random_number: int
    third_random_number: int
    random_number_list: Annotated[list[int],operator.add]  # REDUCER FUNCTION
    average : int

In [5]:
def first_number(state:randomState):
    prompt="Generate a random integer between 10 and 30"
    response=struct_model.invoke(prompt).random_number
    return {"first_random_number":response, "random_number_list":[response]}

def second_number(state:randomState):
    prompt="Generate a random integer between 40 and 70"
    response=struct_model.invoke(prompt).random_number
    return {"second_random_number":response, "random_number_list":[response]}


def third_number(state:randomState):
    prompt="Generate a random integer between 80 and 100"
    response=struct_model.invoke(prompt).random_number
    return {"third_random_number":response, "random_number_list":[response]}


def average_numbers(state:randomState):
    avg=sum(state.random_number_list)/3
    return {"average":avg}

In [6]:
graph=StateGraph(randomState)

graph.add_node("first",first_number)
graph.add_node("second",second_number)
graph.add_node("third",third_number)
graph.add_node("average",average_numbers)

graph.add_edge(START,"first")
graph.add_edge(START,"second")
graph.add_edge(START,"third")

graph.add_edge("first","average")
graph.add_edge("second","average")
graph.add_edge("third","average")

graph.add_edge("average",END)
graph.compile()







ValueError: Failed to reach https://mermaid.ink/ API while trying to render your graph. Status code: 502.

To resolve this issue:
1. Check your internet connection and try again
2. Try with higher retry settings: `draw_mermaid_png(..., max_retries=5, retry_delay=2.0)`
3. Use the Pyppeteer rendering method which will render your graph locally in a browser: `draw_mermaid_png(..., draw_method=MermaidDrawMethod.PYPPETEER)`

In [7]:
workflow=graph.compile()
initial_state={
    "first_random_number": None,
    "second_random_number": None,
    "third_random_number": None,
    "random_number_list" :[],
    "average" :None
    }
final_state=workflow.invoke(initial_state)

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-proj-********************************************************************************************************************************************************KKsA. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}